In [1]:
from qiskit import IBMQ
provider = IBMQ.load_account()

In [16]:
import numpy as np
from qiskit import *
from qiskit import Aer
from scipy.optimize import minimize
import pandas as pd
from qiskit.test.mock import *
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from functools import partial
import itertools
import mitiq
from cmaes import CMA
import cma
from qiskit.tools.monitor import job_monitor

In [3]:
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') fr

In [11]:
L = 3

def TrotterEvolveCircuit(dt, nt, init):
    """
    Implements trotter evolution of the Heisenberg hamiltonian using the circuit from https://arxiv.org/pdf/1906.06343.pdf
    :param tf: time to evolve to
    :param nt: number of trotter steps to use
    :param init: initial state for the trotter evolution. Should be another Qiskit circuit
    """

    # def get_angles(a, b, c):
    #     return (np.pi/2 - 2*c, 2*a - np.pi/2, np.pi/2 - 2*b)
    def get_angles(a):
        return (np.pi/2 - 2*a, 2*a - np.pi/2, np.pi/2 - 2*a)

    def N(circ, qb0, qb1):
        circ.rz(-np.pi/2, qb1)
        circ.cnot(qb1, qb0)
        circ.rz(theta, qb0)
        circ.ry(phi, qb1)
        circ.cnot(qb0, qb1)
        circ.ry(lambd, qb1)
        circ.cnot(qb1, qb0)
        circ.rz(np.pi/2, qb0)
        return circ

    theta, phi, lambd = get_angles(-dt/4)
    circ = init

    for i in range(nt):
        # even (odd indices)
        if (L % 2 == 0):
            # UEven
            for i in range(1, L-1, 2): # L for periodic bdy conditions
                circ = N(circ, i, (i+1)%L)
            # UOdd
            for i in range(0, L-1, 2):
                circ = N(circ, i, (i+1)%L)
        else:
            # UEven
            for i in range(1, L, 2):
                circ = N(circ, i, (i+1)%L)
            # UOdd
            for i in range(0, L-1, 2):
                circ = N(circ, i, (i+1)%L)
            # UBdy
            # circ = N(circ, L-1, 0)
    
    return circ

def AnsatzCircuit(params, p):
    """
    Implements HVA ansatz using circuits from https://arxiv.org/pdf/1906.06343.pdf
    :param params: parameters to parameterize circuit
    :param p: depth of the ansatz
    """
    circ = QuantumCircuit(L)

    def get_angles(a):
        return (np.pi/2 - 2*a, 2*a - np.pi/2, np.pi/2 - 2*a)

    def N(cir, angles, qb0, qb1):
        cir.rz(-np.pi/2, qb1)
        cir.cnot(qb1, qb0)
        cir.rz(angles[0], qb0)
        cir.ry(angles[1], qb1)
        cir.cnot(qb0, qb1)
        cir.ry(angles[2], qb1)
        cir.cnot(qb1, qb0)
        cir.rz(np.pi/2, qb0)
        return cir

    for i in range(p):
        if (L % 2 == 0):
            for j in range(1, L-1, 2): # L for periodic bdy conditions
                circ = N(circ, get_angles(-params[((L-1)*i)+j]/4), j, (j+1)%L)
            for j in range(0, L-1, 2):
                circ = N(circ, get_angles(-params[((L-1)*i)+j]/4), j, (j+1)%L)
        else:
            for j in range(1, L, 2):
                circ = N(circ, get_angles(-params[((L-1)*i)+j]/4), j, (j+1)%L)
            for j in range(0, L-1, 2):
                circ = N(circ, get_angles(-params[((L-1)*i)+j]/4), j, (j+1)%L)
            # circ = N(circ, get_angles(-params[(L*i)+L-1]/4), L-1, 0) # boundary
    return circ

def ReorderBasis(circ):
    """
    Reorders basis so that 0th qubit is on the left side of the tensor product
    :param circ: circuit to reorder, can also be a vector
    """
    if (isinstance(circ, qiskit.circuit.quantumcircuit.QuantumCircuit)):
        for i in range(L//2):
            circ.swap(i, L-i-1)
        return circ
    else:
        perm = np.eye(2**L)
        for i in range(1, 2**L//2):
            perm[:, [i, 2**L-i-1]] = perm[:, [2**L-i-1, i]]
        return perm @ circ

def SimulateAndReorder(circ):
    """
    Executes a circuit using the statevector simulator and reorders basis to match with standard
    """
    circ = ReorderBasis(circ)
    backend = Aer.get_backend('statevector_simulator')
    return execute(circ, backend).result().get_statevector()

def Simulate(circ):
    """
    Executes a circuit using the statevector simulator. Doesn't reorder -- which is needed for intermediate steps in the VTC
    """
    backend = Aer.get_backend('statevector_simulator')
    return execute(circ, backend).result().get_statevector()

def SwapTestCircuit(params, U_v, U_trot, init, p):
    """
    Cost function using the swap test. 
    :param params: parameters new variational circuit that represents U_trot U_v | init >. Need dagger for cost function
    :param U_v: variational circuit that stores the state before the trotter step
    :param U_trot: trotter step
    :param init: initial state
    :param p: number of ansatz steps
    :param shots: number of measurements to take
    """
    U_v_prime = init + AnsatzCircuit(params, p)
    U_v_prime = U_v_prime.qasm()
    U_v_prime = U_v_prime.replace(f'q[{L}]', f'q[{2*L+1}]')

    for i in range(L, -1, -1):
        U_v_prime = U_v_prime.replace(f'q[{i}]', f'q[{i+1}]')
    U_v_prime = circuit.QuantumCircuit.from_qasm_str(U_v_prime)

    comp = init + U_v + U_trot
    comp = comp.qasm()
    comp = comp.replace(f'q[{L}]', f'q[{2*L+1}]')
    for i in range(L, -1, -1):
        comp = comp.replace(f'q[{i}]', f'q[{L+i+1}]')
    comp = circuit.QuantumCircuit.from_qasm_str(comp)

    circ = QuantumCircuit(2*L+1, 1)
    circ.h(0)
    circ += U_v_prime 
    circ += comp

    # controlled swaps
    for i in range(L):
        circ.cswap(0, i+1, L+i+1)
    circ.h(0)
    circ.measure(0,0)

    return circ

def SwapTestExecutor(circuits, backend, shots, filter):
    scale_factors = [1.0, 2.0, 3.0]
    folded_circuits = []
    for circuit in circuits:
        folded_circuits.append([mitiq.zne.scaling.fold_gates_at_random(circuit, scale) for scale in scale_factors])
    folded_circuits = list(itertools.chain(*folded_circuits))

    job = qiskit.execute(
        experiments=folded_circuits,
        backend=backend,
        optimization_level=0,
        shots=shots
    )

    res = job.result()
    if (filter is not None):
        res = filter.apply(res)

    all_counts = [job.result().get_counts(i) for i in range(len(folded_circuits))]
    expectation_values = []
    for counts in all_counts:
        if counts.get('0') is None:
            expectation_values.append(0)
        else:
            expectation_values.append(counts.get('0')/shots)
    
    zero_noise_values = []
    if isinstance(backend, qiskit.providers.aer.backends.qasm_simulator.QasmSimulator): # exact_sim
        for i in range(len(circuits)):
            zero_noise_values.append(np.mean(expectation_values[i*len(scale_factors):(i+1)*len(scale_factors)]))
    else: #device_sim
        fac = mitiq.zne.inference.ExpFactory(scale_factors)
        for i in range(len(circuits)):
            zero_noise_values.append(fac.extrapolate(scale_factors, 
            expectation_values[i*len(scale_factors):(i+1)*len(scale_factors)]))

    return zero_noise_values

    
def SwapTest(params, U_v, U_trot, init, p, backend, shots, filter):
    """

    """
    circs = []
    for param in params:
        circs.append(SwapTestCircuit(param, U_v, U_trot, init, p))
    res = SwapTestExecutor(circs, backend, shots, filter)
    return 1 - np.array(res)

def LoschmidtEchoExecutor(circuits, backend, shots, filter):
    """
    Returns the expectation value to be mitigated.
    :param circuit: Circuit to run.
    :param backend: backend to run the circuit  on
    :param shots: Number of times to execute the circuit to compute the expectation value.
    :param fitter: measurement error mitigator
    """
    scale_factors = [1.0, 2.0, 3.0, 4.0, 5.0]
    folded_circuits = []
    for circuit in circuits:
        folded_circuits.append([mitiq.zne.scaling.fold_gates_at_random(circuit, scale) for scale in scale_factors])
    folded_circuits = list(itertools.chain(*folded_circuits))

    job = qiskit.execute(
        experiments=folded_circuits,
        backend=backend,
        optimization_level=0,
        shots=shots
    )

    job_monitor(job)

    c = c = ['1', '1', '0'] #[str((1 + (-1)**(i+1)) // 2) for i in range(L)]
    c = ''.join(c)[::-1]
    res = job.result()
    if (filter is not None):
        res = filter.apply(res)

    all_counts = [job.result().get_counts(i) for i in range(len(folded_circuits))]
    expectation_values = []
    for counts in all_counts:
        total_allowed_shots = [counts.get(''.join(p)) for p in set(itertools.permutations(c))]
        total_allowed_shots = sum([0 if x is None else x for x in total_allowed_shots])
        if counts.get(c) is None:
            expectation_values.append(0)
        else:
            expectation_values.append(counts.get(c)/total_allowed_shots)
    print(expectation_values)
    # expectation_values = [counts.get(c) / shots for counts in all_counts]
    
    zero_noise_values = []
    if isinstance(backend, qiskit.providers.aer.backends.qasm_simulator.QasmSimulator): # exact_sim
        for i in range(len(circuits)):
            zero_noise_values.append(np.mean(expectation_values[i*len(scale_factors):(i+1)*len(scale_factors)]))
    else: #device_sim
        fac = mitiq.zne.inference.LinearFactory(scale_factors)
        for i in range(len(circuits)):
            zero_noise_values.append(fac.extrapolate(scale_factors, 
            expectation_values[i*len(scale_factors):(i+1)*len(scale_factors)]))
    print(zero_noise_values)
    return zero_noise_values

def LoschmidtEchoCircuit(params, U_v, U_trot, init, p):
    """
    Cost function using the Loschmidt Echo. Just using statevectors currently -- can rewrite using shots
    :param params: parameters new variational circuit that represents U_trot U_v | init >. Need dagger for cost function
    :param U_v: variational circuit that stores the state before the trotter step
    :param U_trot: trotter step
    :param init: initial state
    :param p: number of ansatz steps
    """
    U_v_prime = AnsatzCircuit(params, p)
    circ = init + U_v + U_trot + U_v_prime.inverse()
    circ.measure_all()
    return circ

def LoschmidtEcho(params, U_v, U_trot, init, p, backend, shots, filter):
    """

    """
    circs = []
    for param in params:
        circs.append(LoschmidtEchoCircuit(param, U_v, U_trot, init, p))
    res = LoschmidtEchoExecutor(circs, backend, shots, filter)
    return 1 - np.array(res)

def LoschmidtEchoExact(params, U_v, U_trot, init, p):
    U_v_prime = AnsatzCircuit(params, p)
    circ = init + U_v + U_trot + U_v_prime.inverse()

    circ_vec = Simulate(circ)
    init_vec = Simulate(init)
    return 1 - abs(np.conj(circ_vec) @ init_vec)**2

def CMAES(U_v, U_trot, init, p, backend, shots, filter):
    init_params = np.random.uniform(0, 2*np.pi, (L-1)*p)
    es = cma.CMAEvolutionStrategy(init_params, np.pi/2)
    es.opts.set({'ftarget':1e-11, 'maxiter':100}) #ftarget
    while not es.stop():
        solutions = es.ask()
        es.tell(solutions, LoschmidtEcho(solutions, U_v, U_trot, init, p, backend, shots, filter))
        es.disp()
    return es.result_pretty()

def VTC(tf, dt, p, init):
    """
    :param init: initial state as a circuit
    """
    nt = int(np.ceil(tf / (dt * p)))

    VTCParamList = []
    VTCStepList = [SimulateAndReorder(init.copy())]
    TrotterFixStepList = [init]
    TimeStep = [0]

    for i in range(nt):
        print(i, nt)
        if (i == 0):
            U_v = QuantumCircuit(L)
        else:
            U_v = AnsatzCircuit(VTCParamList[-1], p)
        U_trot = TrotterEvolveCircuit(dt, p, QuantumCircuit(L))
        
        TrotterFixStepList.append(TrotterFixStepList[-1] + U_trot)
        
        init_params = np.random.uniform(0, np.pi, (L-1)*p)
        # res = minimize(fun=SwapTest, x0=init_params, args=(U_v, U_trot, init, p, 8192), method='COBYLA')
        # res = minimize(fun=LoschmidtEcho, x0=init_params, args=(U_v, U_trot, init, p, exact_sim, 8192, None), method='COBYLA')
        # res = minimize(fun=LoschmidtEchoExact, x0=init_params, args=(U_v, U_trot, init, p))
        # res = [0, 1]
        # while (res[1] > 0.05):
        res = CMAES(U_v, U_trot, init, p, device_sim, 8192, meas_fitter.filter)

        print(res)
        
        VTCParamList.append(res[0]) #res.x
        VTCStepList.append(SimulateAndReorder(init + AnsatzCircuit(res[0], p))) #res.x
        TimeStep.append(TimeStep[-1]+(dt*p))
    
    TrotterFixStepList = pd.DataFrame(np.array([SimulateAndReorder(c.copy()) for c in TrotterFixStepList]), index=np.array(TimeStep))
    VTCParamList = pd.DataFrame(np.array(VTCParamList), index=np.array(TimeStep[1:]))
    VTCStepList = pd.DataFrame(np.array(VTCStepList), index=np.array(TimeStep))

    VTCStepList.to_csv(f'./VTD_results/VTD_results_{tf}_{L}_{p}_{dt}.csv')


In [24]:
L = 3
p = 2

qr = QuantumRegister(L)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')

device_backend = FakeSantiago()
device_sim = AerSimulator.from_backend(device_backend)
exact_sim = QasmSimulator(method='statevector_gpu')
sim = device_sim

# Execute the calibration circuits without noise
t_qc = transpile(meas_calibs, sim)
qobj = assemble(t_qc, shots=10000)
cal_results = sim.run(qobj, shots=10000).result()
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
# np.around(meas_fitter.cal_matrix, decimals=2)

real_device = provider.get_backend('ibmq_santiago')

AerError: "Invalid simulation method statevector_gpu. Available methods are: ['automatic', 'statevector', 'density_matrix', 'stabilizer', 'matrix_product_state', 'extended_stabilizer']"

In [23]:
QasmSimulator.from_backend(real_device)

QasmSimulator('qasm_simulator(ibmq_santiago)',
              noise_model=<NoiseModel on ['id', 'measure', 'cx', 'sx', 'reset', 'x']>))

In [8]:
init_params = np.random.uniform(0, 2*np.pi, (L-1)*p)

init = QuantumCircuit(L)
c = ['1', '1', '0'] # [str((1 + (-1)**(i+1)) // 2) for i in range(L)]
for q in range(len(c)):
    if (c[q] == '1'):
        init.x(q)

U_v = QuantumCircuit(L)
U_trot = TrotterEvolveCircuit(1, 2, QuantumCircuit(L))

In [14]:
es = cma.CMAEvolutionStrategy(init_params, np.pi/2) #, {'ftarget':5e-3})
for i in range(1):
    solutions = [np.array([-1.58225511,  0.74729376,  2.41167342, -0.82098098]), np.array([-1.37902519,  2.89376609,  4.26053728, -3.62842058]), np.array([-1.37058037,  3.34519582,  3.88899875,  0.83314033]), np.array([-0.30461341,  1.81845646,  1.84196115,  1.56376546]), np.array([-2.3819801 ,  2.15322492,  3.08738968,  1.63862417]), np.array([3.0144182 , 2.92459152, 3.43306244, 4.24609314]), np.array([2.66645493, 2.46622765, 5.84908148, 1.99691028]), np.array([1.45997549, 1.85978072, 3.08566001, 1.63618961])]
    print(repr(solutions))
    LoschmidtEcho(solutions, U_v, U_trot, init, p, real_device, 8192, meas_fitter.filter)
    # es.logger.add()
    # es.disp()

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=858003, Tue Jun 15 15:51:56 2021)
[array([-1.58225511,  0.74729376,  2.41167342, -0.82098098]), array([-1.37902519,  2.89376609,  4.26053728, -3.62842058]), array([-1.37058037,  3.34519582,  3.88899875,  0.83314033]), array([-0.30461341,  1.81845646,  1.84196115,  1.56376546]), array([-2.3819801 ,  2.15322492,  3.08738968,  1.63862417]), array([3.0144182 , 2.92459152, 3.43306244, 4.24609314]), array([2.66645493, 2.46622765, 5.84908148, 1.99691028]), array([1.45997549, 1.85978072, 3.08566001, 1.63618961])]
Job Status: job has successfully run
[0.3954200935333011, 0.4047144152311877, 0.42878820724040756, 0.44391227627930424, 0.40098010954165464, 0.10036641707822208, 0.23778501628664495, 0.3272406958819643, 0.37992831541218636, 0.36476769911504425, 0.3115918630933161, 0.31865828092243187, 0.3618464231021658, 0.3020429273338505, 0.30928427709111816, 0.4746551170997754, 0.4183220829315333, 0.33535762483130904, 0.36286157024793386, 0.38

In [12]:
U_v = QuantumCircuit(L)
U_trot = TrotterEvolveCircuit(1, 2, QuantumCircuit(L))
es = cma.CMAEvolutionStrategy(init_params, np.pi/2) #, {'ftarget':5e-3})
for i in range(1):
    solutions = [np.array([-1.58225511,  0.74729376,  2.41167342, -0.82098098]), np.array([-1.37902519,  2.89376609,  4.26053728, -3.62842058]), np.array([-1.37058037,  3.34519582,  3.88899875,  0.83314033]), np.array([-0.30461341,  1.81845646,  1.84196115,  1.56376546]), np.array([-2.3819801 ,  2.15322492,  3.08738968,  1.63862417]), np.array([3.0144182 , 2.92459152, 3.43306244, 4.24609314]), np.array([2.66645493, 2.46622765, 5.84908148, 1.99691028]), np.array([1.45997549, 1.85978072, 3.08566001, 1.63618961])]
    LoschmidtEcho(solutions, U_v, U_trot, init, p, device_sim, 8192, meas_fitter.filter)
    # es.logger.add()
    # es.disp()

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=949703, Tue Jun 15 15:50:52 2021)
Job Status: job has successfully run
[0.3546316557257476, 0.3727598566308244, 0.3760730593607306, 0.40028635712824706, 0.4092778793418647, 0.03641618497109827, 0.06434095770939609, 0.10228728270814273, 0.12303980699638119, 0.1508099475245266, 0.24538258575197888, 0.24908302767589197, 0.24034892353377876, 0.25364372469635627, 0.24886980108499096, 0.4964456695198027, 0.4723022192105709, 0.45509977827050996, 0.4387279724529066, 0.41731384193695753, 0.28894399066511084, 0.28919100369251427, 0.282883874518437, 0.2874800637958533, 0.2989506586291583, 0.3891554004942579, 0.38407975945561007, 0.37137614678899084, 0.3620057072971871, 0.34884238646482635, 0.7319721980886186, 0.6996991978609626, 0.6476571013439884, 0.6164602477027566, 0.5669432608208118, 0.03574022955106785, 0.061180124223602486, 0.09557879288203999, 0.10938792390405294, 0.13382649630127774]
[0.34156007731858573, 0.009132923663756407, 0.2440

In [13]:
U_v = QuantumCircuit(L)
U_trot = TrotterEvolveCircuit(1, 2, QuantumCircuit(L))

es = cma.CMAEvolutionStrategy(init_params, np.pi/2) #, {'ftarget':5e-3})
for i in range(1):
    solutions = [np.array([-1.58225511,  0.74729376,  2.41167342, -0.82098098]), np.array([-1.37902519,  2.89376609,  4.26053728, -3.62842058]), np.array([-1.37058037,  3.34519582,  3.88899875,  0.83314033]), np.array([-0.30461341,  1.81845646,  1.84196115,  1.56376546]), np.array([-2.3819801 ,  2.15322492,  3.08738968,  1.63862417]), np.array([3.0144182 , 2.92459152, 3.43306244, 4.24609314]), np.array([2.66645493, 2.46622765, 5.84908148, 1.99691028]), np.array([1.45997549, 1.85978072, 3.08566001, 1.63618961])]
    LoschmidtEcho(solutions, U_v, U_trot, init, p, exact_sim, 8192, None)
    # es.logger.add()
    # es.disp()

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=982282, Tue Jun 15 15:51:08 2021)
Job Status: job has successfully run
[0.3465576171875, 0.34033203125, 0.3516845703125, 0.349365234375, 0.3509521484375, 0.00244140625, 0.0018310546875, 0.0018310546875, 0.0020751953125, 0.002685546875, 0.23974609375, 0.2335205078125, 0.2451171875, 0.2415771484375, 0.2325439453125, 0.5128173828125, 0.513916015625, 0.5086669921875, 0.502685546875, 0.515380859375, 0.28076171875, 0.28857421875, 0.2933349609375, 0.2857666015625, 0.27734375, 0.4063720703125, 0.391357421875, 0.4041748046875, 0.397705078125, 0.4052734375, 0.7916259765625, 0.7900390625, 0.7994384765625, 0.79296875, 0.8072509765625, 0.0069580078125, 0.0076904296875, 0.0072021484375, 0.0079345703125, 0.01025390625]
[0.3477783203125, 0.0021728515625, 0.2385009765625, 0.510693359375, 0.28515625, 0.4009765625, 0.7962646484375, 0.0080078125]
